In [1]:
import math
from scipy.constants import Planck, alpha, e, epsilon_0, speed_of_light, m_p, m_n, c
from random import choices
import simpy as simp
import random as rm
import numpy as np

In [66]:
#function for caculating the electric dipole capture
def cap_E(l,R,v):
    M = 1
    h = Planck/(2*math.pi)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Qlmp = ((ke*h)/(M*c))*Qlm
    w = 2*math.pi*v
    E__amp = -((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Qlm+Qlmp))
    Ue = (c/(2*math.pi*(ke**2)))*(abs(E__amp)**2)
    return sig_cap_lm, Ue

#function for calculating the magnetic dipole caputure    
def cap_M(l,R,v):
    M = 1
    h = Planck/(2*math.pi)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Mlm = Qlm*(v/c)
    Mlmp = (math.sqrt((10*(h/(M*c*R))**2)*(abs(Qlm)**2)))+Mlm
    w = 2*math.pi*v
    M__amp = +((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Mlm+Mlmp))
    Um = (c/(2*math.pi*(ke**2)))*(abs(M__amp)**2)
    return sig_cap_lm, Um

In [139]:
counts = 5000
Type = 'hw+D'
def neutron_collisions(env,counts,Type):
    M = 1
    E = 0
    Me = 0
    scat = 0
    testNum = 0
    l = 1
    A = 2
    R = (A**(1/3))*(10**-13)
    h = Planck/(math.pi*2)
    #for the formation of a duetron by neutron, proton capture
    if Type=='n+p':
        for i in range(counts+1):
            print('Test %d' % testNum)
            #neutron energy is randomly chosen from the range 8.5*10^10 and 10^11
            e = rm.randint(8.5*(10**10),1*(10**11))
            yield env.timeout(1)
            h = Planck/(math.pi*2)
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            cap__E,UE = cap_E(l,R,v)
            cap__M,UM = cap_M(l,R,v)
            react = ['cap E','cap M','scat']
            cross = [cap__E/(2*l +1),cap__M/(2*l + 1),1-((cap__E/(2*l +1))-(cap__M/(2*l + 1)))]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e)
            print(win)
            print(cross)
            if win==['cap E']:
                E+=1
                print(UE,UE)
            if win==['scat']:
                scat+=1
            elif win==['cap M']:
                Me+=1
                print(UE,UE)
            print(E,Me,scat,'\n')
            testNum+=1
    #for photodisitegration of the deutron
    elif Type=='hw+D':
        E = 0
        Me = 0
        scat = 0
        for i in range(counts+1):
            #photon energy is randomly chosen from this range:
            e = rm.randint(10,50)/(1*(10**44.7))
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            ke = ((2*math.pi*v)/c)
            cap__E,UE = cap_E(l,R,v)
            cap__M,UM = cap_M(l,R,v)
            dis_E = cap__E*((k/ke)**2)
            dis_M = cap__M*((k/ke)**2)
            react = ['dis E','dis M','scat']
            cross = [dis_E/(2*l +1),dis_M/(2*l + 1),1-((dis_E/(2*l +1))+(dis_M/(2*l + 1)))]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e)
            print(win)
            print(cross)
            if win==['dis E']:
                E+=1
                print(UE,UM)
            elif win==['dis M']:
                Me+=1
                print(UE,UM)
            elif win==['scat']:
                scat+=1
            print(E,Me,scat,'\n')
env = simp.Environment()
env.process(neutron_collisions(env,counts,Type))
env.run(until=counts+2)

4.78862955592528e-44
['scat']
[0.20363894622185877, 2.392504554886795e-30, 0.7963610537781413]
0 0 1 

6.185313176403487e-44
['scat']
[0.29894171327725577, 3.631348629245611e-30, 0.7010582867227442]
0 0 2 

6.783891870894147e-44
['scat']
[0.3433700043753325, 4.2248945587768594e-30, 0.6566299956246675]
0 0 3 

9.57725911185056e-44
['dis E']
[0.5759779191486362, 7.463611287050674e-30, 0.42402208085136384]
2.329507270036608e-163 2.329507270036608e-163
1 0 3 

1.9952623149688666e-44
['scat']
[0.05477015606642269, 5.867910003260044e-31, 0.9452298439335773]
1 0 4 

5.586734481912827e-44
['dis E']
[0.25661440127572854, 3.0749327006782894e-30, 0.7433855987242715]
7.92679557165234e-164 7.92679557165234e-164
2 0 4 

7.18294433388792e-44
['scat']
[0.3741086325012138, 4.640800746708066e-30, 0.6258913674987863]
2 0 5 

7.382470565384807e-44
['dis E']
[0.389804245210468, 4.8547314738797156e-30, 0.610195754789532]
1.3841560124479667e-163 1.3841560124479667e-163
3 0 5 

7.581996796881693e-44
['scat']


29 0 97 

6.58436563939726e-44
['dis E']
[0.32833326716878924, 4.022974528411915e-30, 0.6716667328312107]
1.1010561706032389e-163 1.1010561706032389e-163
30 0 97 

9.976311574844334e-44
['dis E']
[0.612348960513968, 7.987051905003614e-30, 0.38765103948603197]
2.527677159327916e-163 2.527677159327916e-163
31 0 97 

5.9857869449066e-44
['scat']
[0.2845940791365625, 3.4417061752168344e-30, 0.7154059208634376]
31 0 98 

8.779154185863013e-44
['scat']
[0.5055021338234932, 6.461513126336915e-30, 0.4944978661765068]
31 0 99 

2.39431477796264e-44
['scat']
[0.07199723989357952, 7.843079669600278e-31, 0.9280027601064205]
31 0 100 

1.9952623149688666e-44
['scat']
[0.05477015606642269, 5.867910003260044e-31, 0.9452298439335773]
31 0 101 

5.586734481912827e-44
['dis E']
[0.25661440127572854, 3.0749327006782894e-30, 0.7433855987242715]
7.92679557165234e-164 7.92679557165234e-164
32 0 101 

7.78152302837858e-44
['dis E']
[0.4218333277082964, 5.294418053109188e-30, 0.5781666722917036]
1.53783878373

72 0 189 

7.981049259875467e-44
['scat']
[0.4381612485313815, 5.5201249132819605e-30, 0.5618387514686185]
72 0 190 

5.586734481912827e-44
['scat']
[0.25661440127572854, 3.0749327006782894e-30, 0.7433855987242715]
72 0 191 

7.581996796881693e-44
['scat']
[0.4057134200335098, 5.072612235876632e-30, 0.5942865799664903]
72 0 192 

2.9928934724533e-44
['scat']
[0.10061920162150213, 1.1206235588767671e-30, 0.8993807983784978]
72 0 193 

6.783891870894147e-44
['dis E']
[0.3433700043753325, 4.2248945587768594e-30, 0.6566299956246675]
1.1687979184732282e-163 1.1687979184732282e-163
73 0 193 

4.3895770929315066e-44
['scat']
[0.1787219933654308, 2.0775458246783597e-30, 0.8212780066345692]
73 0 194 

4.988155787422167e-44
['scat']
[0.2164980512159802, 2.5566178129780706e-30, 0.7835019487840198]
73 0 195 

2.5938410094595266e-44
['scat']
[0.08118186176851347, 8.912052607463373e-31, 0.9188181382314865]
73 0 196 

6.185313176403487e-44
['scat']
[0.29894171327725577, 3.631348629245611e-30, 0.70105

131 0 307 

7.382470565384807e-44
['scat']
[0.389804245210468, 4.8547314738797156e-30, 0.610195754789532]
131 0 308 

7.981049259875467e-44
['scat']
[0.4381612485313815, 5.5201249132819605e-30, 0.5618387514686185]
131 0 309 

4.988155787422167e-44
['scat']
[0.2164980512159802, 2.5566178129780706e-30, 0.7835019487840198]
131 0 310 

8.779154185863013e-44
['dis E']
[0.5055021338234932, 6.461513126336915e-30, 0.4944978661765068]
1.957433192183538e-163 1.957433192183538e-163
132 0 310 

1.9952623149688666e-44
['scat']
[0.05477015606642269, 5.867910003260044e-31, 0.9452298439335773]
132 0 311 

9.976311574844334e-44
['dis E']
[0.612348960513968, 7.987051905003614e-30, 0.38765103948603197]
2.527677159327916e-163 2.527677159327916e-163
133 0 311 

7.78152302837858e-44
['scat']
[0.4218333277082964, 5.294418053109188e-30, 0.5781666722917036]
133 0 312 

9.976311574844334e-44
['scat']
[0.612348960513968, 7.987051905003614e-30, 0.38765103948603197]
133 0 313 

3.1924197039501866e-44
['scat']
[0.1

['scat']
[0.48836739554911146, 6.220424895783744e-30, 0.5116326044508885]
181 0 431 

8.579627954366127e-44
['scat']
[0.48836739554911146, 6.220424895783744e-30, 0.5116326044508885]
181 0 432 

9.178206648856787e-44
['dis E']
[0.540356930450522, 6.955050862496204e-30, 0.459643069549478]
2.1394259476551453e-163 2.1394259476551453e-163
182 0 432 

9.377732880353673e-44
['scat']
[0.5580726836595571, 7.207462357936036e-30, 0.4419273163404429]
182 0 433 

5.786260713409714e-44
['scat']
[0.2704836232033879, 3.256222262797474e-30, 0.7295163767966121]
182 0 434 

8.579627954366127e-44
['scat']
[0.48836739554911146, 6.220424895783744e-30, 0.5116326044508885]
182 0 435 

8.9786804173599e-44
['dis E']
[0.5228327082763755, 6.706394889213086e-30, 0.4771672917236245]
2.0474184990556112e-163 2.0474184990556112e-163
183 0 435 

8.9786804173599e-44
['scat']
[0.5228327082763755, 6.706394889213086e-30, 0.4771672917236245]
183 0 436 

7.581996796881693e-44
['scat']
[0.4057134200335098, 5.072612235876632e-

...

1286 0 2905 

2.7933672409564133e-44
['scat']
[0.0907268916460967, 1.0033490935300395e-30, 0.9092731083539033]
1286 0 2906 

3.1924197039501866e-44
['scat']
[0.1108470022225818, 1.2429245673908575e-30, 0.8891529977774182]
1286 0 2907 

3.1924197039501866e-44
['scat']
[0.1108470022225818, 1.2429245673908575e-30, 0.8891529977774182]
1286 0 2908 

3.59147216694396e-44
['scat']
[0.1322673754710171, 1.5022380455988211e-30, 0.8677326245289829]
1286 0 2909 

6.983418102391034e-44
['scat']
[0.3586295282862111, 4.4308460679913066e-30, 0.6413704717137889]
1286 0 2910 

2.1947885464657533e-44
['scat']
[0.06318776672793665, 6.827873580526833e-31, 0.9368122332720633]
1286 0 2911 

4.988155787422167e-44
['dis E']
[0.2164980512159802, 2.5566178129780706e-30, 0.7835019487840198]
6.319192898319791e-164 6.319192898319791e-164
1287 0 2911 

7.18294433388792e-44
['scat']
[0.3741086325012138, 4.640800746708066e-30, 0.6258913674987863]
1287 0 2912 

2.1947885464657533e-44
['dis E']
[0.06318776672793665, 6.8

1343 0 3042 

3.3919459354470733e-44
['scat']
[0.12139962927492602, 1.3701579649443234e-30, 0.878600370725074]
1343 0 3043 

8.579627954366127e-44
['dis E']
[0.48836739554911146, 6.220424895783744e-30, 0.5116326044508885]
1.8694700270389243e-163 1.8694700270389243e-163
1344 0 3043 

9.178206648856787e-44
['scat']
[0.540356930450522, 6.955050862496204e-30, 0.459643069549478]
1344 0 3044 

9.976311574844334e-44
['scat']
[0.612348960513968, 7.987051905003614e-30, 0.38765103948603197]
1344 0 3045 

7.18294433388792e-44
['scat']
[0.3741086325012138, 4.640800746708066e-30, 0.6258913674987863]
1344 0 3046 

8.9786804173599e-44
['dis E']
[0.5228327082763755, 6.706394889213086e-30, 0.4771672917236245]
2.0474184990556112e-163 2.0474184990556112e-163
1345 0 3046 

7.981049259875467e-44
['dis E']
[0.4381612485313815, 5.5201249132819605e-30, 0.5618387514686185]
1.617713381969865e-163 1.617713381969865e-163
1346 0 3046 

6.384839407900373e-44
['scat']
[0.31352266778315147, 3.825115563946192e-30, 0.6

1372 0 3160 

6.983418102391034e-44
['scat']
[0.3586295282862111, 4.4308460679913066e-30, 0.6413704717137889]
1372 0 3161 

2.39431477796264e-44
['scat']
[0.07199723989357952, 7.843079669600278e-31, 0.9280027601064205]
1372 0 3162 

5.187682018919053e-44
['scat']
[0.2296169798634593, 2.7250895496006422e-30, 0.7703830201365407]
1372 0 3163 

2.9928934724533e-44
['scat']
[0.10061920162150213, 1.1206235588767671e-30, 0.8993807983784978]
1372 0 3164 

2.9928934724533e-44
['scat']
[0.10061920162150213, 1.1206235588767671e-30, 0.8993807983784978]
1372 0 3165 

3.9905246299377333e-44
['scat']
[0.15491339504485205, 1.7806307395057666e-30, 0.845086604955148]
1372 0 3166 

8.579627954366127e-44
['scat']
[0.48836739554911146, 6.220424895783744e-30, 0.5116326044508885]
1372 0 3167 

8.180575491372354e-44
['scat']
[0.45469456546415077, 5.749709710528597e-30, 0.5453054345358492]
1372 0 3168 

9.976311574844334e-44
['dis E']
[0.612348960513968, 7.987051905003614e-30, 0.38765103948603197]
2.5276771593

[0.1787219933654308, 2.0775458246783597e-30, 0.8212780066345692]
1435 0 3292 

2.1947885464657533e-44
['dis E']
[0.06318776672793665, 6.827873580526833e-31, 0.9368122332720633]
1.2233957451147113e-164 1.2233957451147113e-164
1436 0 3292 

8.579627954366127e-44
['scat']
[0.48836739554911146, 6.220424895783744e-30, 0.5116326044508885]
1436 0 3293 

1.9952623149688666e-44
['scat']
[0.05477015606642269, 5.867910003260044e-31, 0.9452298439335773]
1436 0 3294 

6.983418102391034e-44
['scat']
[0.3586295282862111, 4.4308460679913066e-30, 0.6413704717137889]
1436 0 3295 

9.776785343347447e-44
['dis E']
[0.5940706525366495, 7.723480129714713e-30, 0.4059293474633505]
2.42758114381853e-163 2.42758114381853e-163
1437 0 3295 

2.39431477796264e-44
['scat']
[0.07199723989357952, 7.843079669600278e-31, 0.9280027601064205]
1437 0 3296 

2.39431477796264e-44
['scat']
[0.07199723989357952, 7.843079669600278e-31, 0.9280027601064205]
1437 0 3297 

9.776785343347447e-44
['scat']
[0.5940706525366495, 7.7234

5.187682018919053e-44
['dis E']
[0.2296169798634593, 2.7250895496006422e-30, 0.7703830201365407]
6.834839038822681e-164 6.834839038822681e-164
1471 0 3380 

3.7909983984408466e-44
['scat']
[0.1434413552620404, 1.639086358733777e-30, 0.8565586447379596]
1471 0 3381 

9.377732880353673e-44
['scat']
[0.5580726836595571, 7.207462357936036e-30, 0.4419273163404429]
1471 0 3382 

5.786260713409714e-44
['scat']
[0.2704836232033879, 3.256222262797474e-30, 0.7295163767966121]
1471 0 3383 

9.377732880353673e-44
['scat']
[0.5580726836595571, 7.207462357936036e-30, 0.4419273163404429]
1471 0 3384 

7.78152302837858e-44
['dis E']
[0.4218333277082964, 5.294418053109188e-30, 0.5781666722917036]
1.5378387837351033e-163 1.5378387837351033e-163
1472 0 3384 

2.39431477796264e-44
['scat']
[0.07199723989357952, 7.843079669600278e-31, 0.9280027601064205]
1472 0 3385 

6.384839407900373e-44
['scat']
[0.31352266778315147, 3.825115563946192e-30, 0.6864773322168485]
1472 0 3386 

3.9905246299377333e-44
['scat'